In [5]:
# Python requistes
from collections import Counter
from scipy.sparse import csr_matrix, hstack
from tqdm import tqdm

import numpy as np
import time
import pandas as pd
import os
import re

import pickle
import matplotlib.pyplot as plt


# Graph / Visualization
from networkx.drawing.nx_agraph import graphviz_layout
import networkx as nx
import pygraphviz


## Tweet preprocessor
import preprocessor as p

## NLTK tokenization / lemmatization
import nltk

from nltk.tokenize import treebank
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer

from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.wordnet import Synset
from nltk.corpus import sentiwordnet as swn
from nltk.corpus.reader import WordListCorpusReader
from nltk.corpus import opinion_lexicon

from nltk.wsd import lesk
from nltk.stem.wordnet import WordNetLemmatizer
import nltk.sentiment


In [6]:
synset_list = list(wn.all_synsets())

In [10]:
len(synset_list)

117659

In [9]:
cnt = 0
for ss in synset_list:
    df = ss.definition()
    if 'not' in df or 'Not' in df:
        cnt += 1
        print(ss.name())
        print(df)

unable.a.01
(usually followed by `to') not having the necessary means or skill or know-how
unabridged.a.01
(used of texts) not shortened
relative.a.01
estimated by comparison; not absolute or complete
receptive.s.04
able to absorb liquid (not repellent)
nonabsorbent.a.01
not capable of absorbing or soaking up (liquids)
abstinent.s.01
self-restraining; not indulging an appetite especially for food or drink
conceptional.s.01
being of the nature of a notion or concept
concrete.a.01
capable of being perceived by the senses; not abstract or imaginary
rare.s.03
not widely distributed
unabused.a.01
not physically abused; treated properly
unexceptionable.s.01
completely acceptable; not open to exception or reproach
unobjectionable.s.03
not objectionable; it's the means that one can't accept"
unacceptable.a.02
not acceptable; not welcome
inaccessible.a.01
capable of being reached only with great difficulty or not at all
unaccommodating.a.01
not accommodating
inaccurate.a.01
not exact
faulty.s.0

neutral.s.03
not supporting or favoring either side in a war, dispute, or contest
undescriptive.a.01
not successful in describing
preferable.s.01
more desirable than another
undesirable.a.01
not wanted
blotted_out.s.01
reduced to nothingness
scorched.s.02
having everything destroyed so nothing is left salvageable by an enemy
indestructible.a.01
not easily destroyed
undestroyable.s.02
not capable of being destroyed
indeterminable.a.01
not capable of being definitely decided or ascertained
indeterminate.s.04
not capable of being determined
unascertainable.s.01
not able to be ascertained; resisting discovery
indeterminate.a.01
not precisely determined or established; not fixed or known in advance
determinate.a.02
not continuing to grow indefinitely at the apex
undeveloped.a.01
not developed
vestigial.s.01
not fully developed in mature animals
undifferentiated.a.01
not differentiated
difficult.a.01
not easy; requiring great physical or mental effort to accomplish or comprehend or endure
se

lacking in reality or substance or genuineness; not corresponding to acknowledged facts or criteria
fanciful.s.02
not based on fact; unreal
proper.s.02
having all the qualities typical of the thing specified; not just a snack"
unreal.a.02
not actually such; being or seeming fanciful or imaginary
deceptive.s.01
causing one to believe what is not true or fail to believe what is true
nominal.a.04
of, relating to, or characteristic of an amount that is not adjusted for inflation
unrealistic.a.01
not realistic
unreasonable.a.01
not reasonable; not showing good judgment
mindless.s.05
not marked by the use of reason
nonreciprocal.a.01
not reciprocal
nonreciprocating.s.01
not moving alternately forward and backward
unanswered.s.01
not returned in kind
unrefined.a.02
(used of persons and their behavior) not refined; uncouth
ungentlemanly.s.01
not befitting a gentleman
unprocessed.a.02
not altered from an original or natural state
unvulcanized.s.01
(used of rubber) not subjected to the process o

the act of postponing to another time or place
antagonism.n.04
(biochemistry) interference in or inhibition of the physiological action of a chemical substance by another having a similar structure
conversion.n.08
act of exchanging one type of money or security for another
circularization.n.01
circulating printed notices as a means of advertising
alienation.n.03
(law) the voluntary and absolute transfer of title and possession of real property from one person to another
conveyance.n.04
act of transferring property title from one person to another
quitclaim.n.02
act of transferring a title or right or claim to another
delivery.n.04
the voluntary transfer of something (title or possession) from one party to another
retail.n.01
the selling of goods to consumers; usually in small quantities and not for resale
tribute.n.02
payment by one nation for protection by another
migration.n.01
the movement of persons from one country or locality to another
misconduct.n.01
bad or dishonest management

supertonic.n.01
(music) the second note of a diatonic scale
mediant.n.01
(music) the third note of a diatonic scale; midway between the tonic and the dominant
subdominant.n.01
(music) the fourth note of the diatonic scale
dominant.n.01
(music) the fifth note of the diatonic scale
submediant.n.01
(music) the sixth note of a major or minor scale (or the third below the tonic)
subtonic.n.01
(music) the seventh note of the diatonic scale
pedal_point.n.01
a sustained bass note
interval.n.04
the difference in pitch between two notes
third.n.04
the musical interval between one note and another three notes away from it
fourth.n.03
the musical interval between one note and another four notes away from it
fifth.n.04
the musical interval between one note and another five notes away from it
sixth.n.03
the musical interval between one note and another six notes away from it
seventh.n.03
the musical interval between one note and another seven notes away from it
trill.n.01
a note that alternates rapi

aloeaceae.n.01
one of many families or subfamilies into which some classification systems subdivide the Liliaceae but not widely accepted
alstroemeriaceae.n.01
one of many families or subfamilies into which some classification systems subdivide the Liliaceae but not widely accepted; sometimes included in subfamily Amaryllidaceae
aphyllanthaceae.n.01
one of many families or subfamilies into which some classification systems subdivide the Liliaceae but not widely accepted; includes genus Aphyllanthes
colchicaceae.n.01
one of many subfamilies into which some classification systems subdivide the Liliaceae but not widely accepted: genera Colchicum and Gloriosa
hemerocallidaceae.n.01
one of many subfamilies into which some classification systems subdivide the Liliaceae but not widely accepted; includes genus Hemerocallis
hostaceae.n.01
one of many families or subfamilies into which some classification systems subdivide the Liliaceae but not widely accepted; includes genus Hosta
hyacinthaceae

In [2]:
wordnet_graph_synset = nx.Graph(engine='sfdp', pack=True)

seen = set()
for ss in tqdm(synset_list):
    wordnet_graph_synset.add_node(ss.name())
    for lm in ss.lemmas():
        _lm = lm.name()
        if not _lm in seen:
            seen.add(_lm)
            wordnet_graph_synset.add_node(_lm)
        wordnet_graph_synset.add_edge(_lm, ss.name())

path = './wordnet_graph_synset.p'
nx.write_gpickle(wordnet_graph_synset, path)

100%|██████████| 117659/117659 [00:01<00:00, 82730.46it/s]


In [3]:
wordnet_graph_synset_cleaned = nx.Graph(engine='sfdp', pack=True)

for ss in tqdm(synset_list):
    wordnet_graph_synset_cleaned.add_node(ss.name())

for ss in tqdm(synset_list):
    nb_list = [k for m in [n for n in wordnet_graph_synset.neighbors(ss.name())] for k in wordnet_graph_synset.neighbors(m)]    
    for nb in nb_list:
        if wordnet_graph_synset_cleaned.has_edge(ss.name(), nb) == False:
            wordnet_graph_synset_cleaned.add_edge(ss.name(), nb)

path = './wordnet_graph_synset_cleaned.p'
nx.write_gpickle(wordnet_graph_synset_cleaned, path)

100%|██████████| 117659/117659 [00:02<00:00, 57002.10it/s]


In [3]:
df = wn.synset('amazing.s.02').definition()

In [4]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import treebank
from nltk.tokenize import TweetTokenizer

In [16]:
df

'inspiring awe or admiration or wonder'

In [11]:
word_tokenize(df)

['inspiring', 'awe', 'or', 'admiration', 'or', 'wonder']

In [15]:
tb_tkn = treebank.TreebankWordTokenizer()
tb_tkn.tokenize(df)

['inspiring', 'awe', 'or', 'admiration', 'or', 'wonder']

In [30]:
tw_tkn = TweetTokenizer()
tw_tkn.tokenize(df)

['inspiring', 'awe', 'or', 'admiration', 'or', 'wonder']

In [27]:
# Negation

replace_dict = {
    "don't": "do not",
    "won't": "will not",
    "didn't": "did not",
    "doesn't": "does not",
    "can't": "can not",
    "couldn't": "could not",
    "isn't": "is not",
    "shouldn't": "should not",
    "wouldn't": "would not",
    "wasn't": "was not",
    "weren't": "were not",
    "haven't": "have not",
    "ain't": "is not",
    "aren't": "are not",
}

def replace_word(text):
    for word in replace_dict:
        if word in text:  # Small first letter
            text = text.replace(word, replace_dict[word])
        elif word[0].title() + word[1:] in text:  # Big first letter
            text = text.replace(word[0].title() + word[1:],
                                replace_dict[word][0].title() + replace_dict[word][1:])

    return text

def neg_tagging(word_list):
    string = ' '.join(word_list)
    transformed = re.sub(r'\b(?:not|never|no)\b[\w\s]+[^\w\s]', 
           lambda match: re.sub(r'(\s+)(\w+)', r'\1NEG_\2', match.group(0)), 
           string,
           flags=re.IGNORECASE)
    
    return transformed

In [28]:
def negate(word_list):
    negged_sentence = neg_tagging(word_list)
    negged_tokens = negged_sentence.split()

    tokens = []
    token_pair_list = []
    
    for word in negged_tokens:
        negation = False
        if word.startswith('NEG_'):
            negation = True
            word = word[4:]
        token_pair_list.append((word, negation))
        tokens.append(word)
    
    return tokens, token_pair_list

In [31]:
df_tk = word_tokenize(replace_word(df))

In [32]:
df_tk

['inspiring', 'awe', 'or', 'admiration', 'or', 'wonder']

In [34]:
from pywsd.lesk import simple_lesk, original_lesk, cosine_lesk, adapted_lesk
from pywsd import disambiguate
from pywsd.similarity import max_similarity

In [37]:
df_pair = disambiguate(df, cosine_lesk)

In [38]:
df_pair

[('inspiring', Synset('inspire.v.01')),
 ('awe', Synset('awe.n.01')),
 ('or', None),
 ('admiration', Synset('admiration.n.01')),
 ('or', None),
 ('wonder', Synset('wonder.n.01'))]

In [40]:
synset_list = list(wn.all_synsets())

In [45]:
from tqdm import tqdm

In [7]:
# Better WSD library
from pywsd.lesk import simple_lesk, original_lesk, cosine_lesk, adapted_lesk
from pywsd import disambiguate
from pywsd.similarity import max_similarity

In [ ]:
all_pairs_from_definition = []
for ss in tqdm(synset_list):
    df = ss.definition()
    df_pair = disambiguate(df, cosine_lesk)
    all_pairs_from_definition.extend(df_pair)

 25%|██▍       | 29110/117659 [43:31<2:12:24, 11.15it/s]

In [27]:
all_pairs_from_definition = []
for ss in tqdm([wn.synset('amazing.s.02'), wn.synset('good.a.01')]):
    df = ss.definition()
    curr_df_pair_list = disambiguate(df, cosine_lesk)
    df_pair_txt_list = []
    for curr_df_pair in curr_df_pair_list:
        if curr_df_pair[1] is None:
            df_pair_txt_list.append(curr_df_pair)
        else:
            df_pair_txt_list.append((curr_df_pair[0], curr_df_pair[1].name()))
    all_pairs_from_definition.append((ss.name(), df_pair_txt_list))

100%|██████████| 2/2 [00:00<00:00, 10.21it/s]


In [28]:
all_pairs_from_definition

[('amazing.s.02',
  [('inspiring', 'inspire.v.01'),
   ('awe', 'awe.n.01'),
   ('or', None),
   ('admiration', 'admiration.n.01'),
   ('or', None),
   ('wonder', 'wonder.n.01')]),
 ('good.a.01',
  [('having', None),
   ('desirable', 'desirable.a.01'),
   ('or', None),
   ('positive', 'positive.a.01'),
   ('qualities', 'quality.n.01'),
   ('especially', 'particularly.r.01'),
   ('those', None),
   ('suitable', None),
   ('for', None),
   ('a', None),
   ('thing', 'thing.n.01'),
   ('specified', 'stipulate.v.01')])]

In [29]:
import pickle

In [30]:
pickle.dump(all_pairs_from_definition, open( "all_pairs_from_definition.p", "wb" ) )